# OpenMM Introduction

## User Guide
* [OpenMM Users Manual and Theory Guide](http://docs.openmm.org/latest/userguide/index.html) <br>
    The place where you should look first! Good Explanations!
    * Theory: [Standard Forces](http://docs.openmm.org/latest/userguide/theory.html#standard-forces) <br>
        If you want to know the formulas of the used forces.
* [OpenMM Python API](http://docs.openmm.org/latest/api-python/index.html) <br>
    Only the python interface
    
## additional tools
* [OpenMMTools](https://openmmtools.readthedocs.io/en/0.17.0/) <br>
    A lot of additional implementations
* [ParmEd](https://parmed.github.io/ParmEd/html/index.html) <br>
    general tool for aiding in investigations of biomolecular systems
* [choderalab/openmm-forcefields](https://github.com/choderalab/openmm-forcefields) <br>
Conversion tools for and repository of CHARMM and AMBER forcefields for OpenMM.



# How is OpenMM structured?

* Layered structure
    * Public Interface ( **OpenMM Public API** )  <span style='color:red'>**<- thats where we work**</span>
    * Platform Interdependent Code ( OpenMM Implementation Layer )
    * Platform Abstraction Layer ( OpenMM Low Level API )
    * Computational Kernels ( OpenCL/CUDA/MPI/etc. )

## OpenMM Public API ?

* Python or C interface to control OpenMM
* Script like language
    * requires to build the simulation from single components

## The General ClassDiagram in OpenMM
![ClassDiagram](img/ClassDiagram.png)
Just an overview how everything is connected

## Let's focus on the crucial elements

![crucial_parts](img/crucial_elements.png)

# System

## What's a system ?

![system](img/system.png)

## Forces
In a broader sense: `ALL interactions`
    
* non-bonded forces
* bonded forces
* constraints
* ANY kind of user defined `CustomForces`
* <span style='color:red'>thermostat and barostat </span>

### How does this actually look like? 

In [1]:
# Let's import everything
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [2]:
# read the pdb file
pdb = PDBFile('spce.pdb')

In [3]:
# read the force field file
forcefield = ForceField('spce.xml')

**How does this force field file looks like ?**
```xml
<ForceField>
 <AtomTypes>
  <Type name="spce-O" class="OW" element="O" mass="15.99943"/>
  <Type name="spce-H" class="HW" element="H" mass="1.007947"/>
 </AtomTypes>
 <Residues>
  <Residue name="HOH">
   <Atom name="O" type="spce-O"/>
   <Atom name="H1" type="spce-H"/>
   <Atom name="H2" type="spce-H"/>
   <Bond atomName1="O" atomName2="H1"/>
   <Bond atomName1="O" atomName2="H2"/>
  </Residue>
 </Residues>
 <HarmonicBondForce>
  <Bond class1="OW" class2="HW" length="0.1" k="462750.4"/>
 </HarmonicBondForce>
 <HarmonicAngleForce>
  <Angle class1="HW" class2="OW" class3="HW" angle="1.91061193216" k="836.8"/>
 </HarmonicAngleForce>
 <NonbondedForce coulomb14scale="0.833333" lj14scale="0.5">
  <Atom type="spce-O" charge="-0.8476" sigma="0.31657195050398818" epsilon="0.6497752"/>
  <Atom type="spce-H" charge="0.4238" sigma="1" epsilon="0"/>
 </NonbondedForce>
</ForceField>
```

create the system

In [4]:
system = forcefield.createSystem(pdb.topology, 
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer,
                                 constraints=HAngles)

**constraints**

|Value|Meaning|
|-----|-------|
| None	| No constraints are applied. This is the default value. |
| HBonds	| The lengths of all bonds that involve a hydrogen atom are constrained. |
| AllBonds	| The lengths of all bonds are constrained. |
| HAngles	| The lengths of all bonds are constrained. In addition, all angles of the form H-X-H or H-O-X (where X is an arbitrary atom) are constrained. |


# Integrator

"Just" a integrator. Some integrators also can adjust temperatures and work as thermostat simultaneously.

In [5]:
# LangevinIntegrator(temperature,  friction coefficient, step size)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond,0.002*picoseconds)

## Example for adding a `integrator` + `thermostat`

```python
system.addForce(AndersenThermostat(300*kelvin, 1/picosecond))
integrator = VerletIntegrator(0.002*picoseconds)
```

a `thermostat` is a `Force` so it belongs to the `system`

## Example for adding a barostat

```python
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
```
<p style='color:red; font-weight:bold'>Only MonteCarloBarostat available!</p>

# Simulations
![simulation](img/simulation.png)

Let's create a simulation

In [6]:
simulation = Simulation(pdb.topology, system, integrator)

In [7]:
# set positions
simulation.context.setPositions(pdb.positions)

# Recap
## What did we do till now?


```python
# Let's import everything
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

# read the pdb file
pdb = PDBFile('spce.pdb')

# read the force field file
forcefield = ForceField('spce.xml')

# define a integrator (with integrated thermostat)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond,0.002*picoseconds)

# create a simulation
simulation = Simulation(pdb.topology, system, integrator)

# set positions
simulation.context.setPositions(pdb.positions)
```

## What did we got?

![simulation_context](img/simulation_context.png)

# Reporters

In [8]:
# report thermodynamical properties
simulation.reporters.append(StateDataReporter('thermo.csv', 1000,
                                              step=True, potentialEnergy=True, temperature=True))

In [9]:
# report positions
simulation.reporters.append(DCDReporter('trajectory.dcd', 1000))

## Overview reporters

<div style='width:100%; float:left'>
    <div style='width:49%; float:left'>
        <h5>OpenMM</h5>
        <ul>
            <li> CheckpointReporter </li>         
            <li> DCDReporter </li>
            <li> PDBReporter </li>
            <li> PDBxReporter </li>
            <li> StateDataReporter</li>
        </ul>
    </div>
    <div style='width:49%; float:left'>
        <h5>MDTraj</h5>
        <ul>
            <li> HDF5Reporter </li>         
            <li> NetCDFReporter </li>
            <li> DCDReporter </li>
        </ul>
    </div>
</div>

<div style='width:100%; float:left'>
    <div style='width:49%; float:left'>
        <h5></h5>
    </div>
    <div style='width:49%; float:left'>
        <h5>ParmED</h5>
        <ul>
            <li> StateDataReporter </li>         
            <li> NetCDFReporter </li>
            <li> MdcrdReporter </li>
            <li> RestartReporter </li>
            <li> ProgressReporter</li>
        </ul>
    </div>
</div>


# Energy mimization

In [10]:
# simulation.minimizeEnergy(tolerance=10*kilojoule/mole, maxIterations=0)
simulation.minimizeEnergy()

# Do a simulation

In [11]:
# Let's set some Velocities
simulation.context.setVelocitiesToTemperature(300*kelvin)

In [12]:
# Let's simulate
simulation.step(1000)

# The whole picture

```python
# Let's import everything
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

# read the pdb file
pdb = PDBFile('spce.pdb')
# read the force field file
forcefield = ForceField('spce.xml')
# define a integrator (with integrated thermostat)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond,0.002*picoseconds)
# create a simulation
simulation = Simulation(pdb.topology, system, integrator)
# set positions
simulation.context.setPositions(pdb.positions)
# report thermodynamical properties
simulation.reporters.append(StateDataReporter('thermo.csv', 1000, step=True, potentialEnergy=True, temperature=True))
# report positions
simulation.reporters.append(DCDReporter('trajectory.dcd', 1000))
# minimize the system
simulation.minimizeEnergy()
# generate velocities
simulation.context.setVelocitiesToTemperature(300*kelvin)
# run the simulation
simulation.step(1000)
```

# What did we skip?

* modifying structures before we create a system
* `Platforms`
* getting data from the `simulation` object

# Platforms

* `Reference` <br>
    clean user-readable code (not performance in mind)
* `CPU` <br>
    optimized CPU code (multithreaded - OpenMM)
* `CUDA` <br>
    Nvidia-GPU's
* `OpenCL` <br>
    variety of types of GPU's and CPU's
    
* <span style='color:red; font-weight:900'>no MPI</span>

## How to use a platform?

In [13]:
# Let's import everything
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

# read the pdb file
pdb = PDBFile('spce.pdb')
# read the force field file
forcefield = ForceField('spce.xml')
# define a integrator (with integrated thermostat)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond,0.002*picoseconds)

In [14]:
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(pdb.topology, system, integrator, platform)

How to change properties of a platform?

In [15]:
# Let's import everything
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

# read the pdb file
pdb = PDBFile('spce.pdb')
# read the force field file
forcefield = ForceField('spce.xml')
# define a integrator (with integrated thermostat)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond,0.002*picoseconds)

In [16]:
platform = Platform.getPlatformByName('CUDA')
properties = {'DeviceIndex': '0,1', 'Precision': 'double'}
simulation = Simulation(pdb.topology, system, integrator, platform)

## Precision aka Thing's one should stress


![EnergyDrift](img/EnergyDrift.png)

<p>
<div style='color:red; font-weight:bold; font-size:large;'>ALLWAYS check your default options of your MD engine</div><p>